# Runnable Sequence

In [5]:
import os
from google.colab import userdata

openai_api_key = userdata.get('OPENAI_API_KEY')

# Set the API key as an environment variable
os.environ['OPENAI_API_KEY'] = openai_api_key

In [8]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableSequence
from google.colab import userdata

openai_api_key = userdata.get('OPENAI_API_KEY')

prompt1 = PromptTemplate(
    template='Write a joke about {topic}',
    input_variables=['topic']
)

model = ChatOpenAI(api_key=openai_api_key)

parser = StrOutputParser()

prompt2 = PromptTemplate(
    template='Explain the following joke - {text}',
    input_variables=['text']
)

chain = RunnableSequence(prompt1, model, parser, prompt2, model, parser)

print(chain.invoke({'topic':'AI'}))

This joke plays on the dual meaning of the word "byte." In the context of computers and technology, a byte is a unit of digital information. However, in the joke, "byte" is used as a pun for "bit," which sounds like "bites." The AI broke up with its supercomputer girlfriend because she couldn't handle his small and probably cheesy "byte" sized jokes, meaning his jokes were too corny or not funny enough for her taste.


# Runnable Parallel

In [11]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv
from langchain_core.runnables import RunnableSequence, RunnableParallel # Added RunnableParallel import
from google.colab import userdata # Import userdata

# Load environment variables - this might not be needed if using Colab secrets
# load_dotenv()

# Fetch API key from Colab secrets
openai_api_key = userdata.get('OPENAI_API_KEY')

prompt1 = PromptTemplate(
    template='Generate a tweet about {topic}',
    input_variables=['topic']
)

prompt2 = PromptTemplate(
    template='Generate a Linkedin post about {topic}',
    input_variables=['topic']
)

# Pass the API key to ChatOpenAI
model = ChatOpenAI(api_key=openai_api_key)

parser = StrOutputParser()

parallel_chain = RunnableParallel({
    'tweet': RunnableSequence(prompt1, model, parser),
    'linkedin': RunnableSequence(prompt2, model, parser)
})

result = parallel_chain.invoke({'topic':'AI'})

print("Tweet:")
print(result['tweet'])
print("\nLinkedIn Post:")
print(result['linkedin'])

Tweet:
"Exciting to see how AI continues to revolutionize industries from healthcare to finance. The possibilities are endless! #AI #technology"

LinkedIn Post:
Exciting news in the world of AI! The latest advancements in artificial intelligence are revolutionizing industries and creating new opportunities for innovation and growth. From machine learning to natural language processing, AI is shaping the future of technology. Stay ahead of the curve and embrace the power of AI in your business today. #AI #ArtificialIntelligence #Innovation #TechTrends #FutureOfWork.


# Runnable Passthrough

In [13]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv
from langchain_core.runnables import RunnableSequence, RunnableParallel, RunnablePassthrough # Corrected import path
from google.colab import userdata # Import userdata

# Load environment variables - this might not be needed if using Colab secrets
# load_dotenv()

# Fetch API key from Colab secrets
openai_api_key = userdata.get('OPENAI_API_KEY')

prompt1 = PromptTemplate(
    template='Write a joke about {topic}',
    input_variables=['topic']
)

model = ChatOpenAI(api_key=openai_api_key)

parser = StrOutputParser()

prompt2 = PromptTemplate(
    template='Explain the following joke - {text}',
    input_variables=['text']
)

joke_gen_chain = RunnableSequence(prompt1, model, parser)

parallel_chain = RunnableParallel({
    'joke': RunnablePassthrough(),
    'explanation': RunnableSequence(prompt2, model, parser)
})

final_chain = RunnableSequence(joke_gen_chain, parallel_chain)

print(final_chain.invoke({'topic':'cricket'}))

{'joke': 'Why did the cricket team go to the bank?\n\nTo make a good deposit!', 'explanation': 'This joke plays on the word "deposit," which can mean both adding money to a bank account and the act of placing a cricket ball in a specific spot during a game. In this case, the joke implies that the cricket team went to the bank not to add money, but to make a good deposit by placing the cricket ball in a strategic position on the pitch during the game.'}


# Runnable Lambda

In [15]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableSequence, RunnableLambda, RunnablePassthrough, RunnableParallel # Corrected import path
from google.colab import userdata # Import userdata

# Fetch API key from Colab secrets
openai_api_key = userdata.get('OPENAI_API_KEY')

def word_count(text):
    return len(text.split())

prompt = PromptTemplate(
    template='Write a joke about {topic}',
    input_variables=['topic']
)

# Pass the API key to ChatOpenAI
model = ChatOpenAI(api_key=openai_api_key)

parser = StrOutputParser()

joke_gen_chain = RunnableSequence(prompt, model, parser)

parallel_chain = RunnableParallel({
    'joke': RunnablePassthrough(),
    'word_count': RunnableLambda(word_count)
})

final_chain = RunnableSequence(joke_gen_chain, parallel_chain)

result = final_chain.invoke({'topic':'AI'})

final_result = """{} \n word count - {}""".format(result['joke'], result['word_count'])

print(final_result)

Why did the AI break up with his girlfriend?

He couldn't handle her emotional baggage! 
 word count - 15


# Rnnable Branch

In [18]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv
from langchain_core.runnables import RunnableSequence, RunnableParallel, RunnablePassthrough, RunnableBranch, RunnableLambda # Corrected import path
from google.colab import userdata # Import userdata

# Fetch API key from Colab secrets
openai_api_key = userdata.get('OPENAI_API_KEY')

# load_dotenv() # No longer needed when using userdata

prompt1 = PromptTemplate(
    template='Write a detailed report on {topic}',
    input_variables=['topic']
)

prompt2 = PromptTemplate(
    template='Summarize the following text \n {text}',
    input_variables=['text']
)

model = ChatOpenAI(api_key=openai_api_key) # Pass the API key

parser = StrOutputParser()

report_gen_chain = RunnableSequence(prompt1, model , parser)

branch_chain = RunnableBranch(
    (lambda x: len(x.split())>300, RunnableSequence(prompt2 , model ,parser)),
    RunnablePassthrough()
)

final_chain = RunnableSequence(report_gen_chain, branch_chain)

print(final_chain.invoke({'topic':'Russia vs Ukraine'}))

The relationship between Russia and Ukraine is complex and tumultuous, with tensions escalating since the 2014 annexation of Crimea by Russia. The annexation led to a sharp deterioration in relations and sparked the War in Donbass between pro-Russian separatists and the Ukrainian government. The conflict has caused thousands of deaths and strained diplomatic relations between the two countries. The international community has called for a peaceful resolution, but progress has been slow. The situation is further complicated by Russia's presence in Crimea and ongoing war of words. It is crucial for both sides to work towards a peaceful resolution to avoid further bloodshed and instability.


#LECL

In [17]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv
from langchain_core.runnables import RunnableSequence, RunnableParallel, RunnablePassthrough, RunnableBranch, RunnableLambda # Corrected import path
from google.colab import userdata # Import userdata

# Fetch API key from Colab secrets
openai_api_key = userdata.get('OPENAI_API_KEY')

# load_dotenv() # No longer needed when using userdata

prompt1 = PromptTemplate(
    template='Write a detailed report on {topic}',
    input_variables=['topic']
)

prompt2 = PromptTemplate(
    template='Summarize the following text \n {text}',
    input_variables=['text']
)

model = ChatOpenAI(api_key=openai_api_key) # Pass the API key

parser = StrOutputParser()

report_gen_chain = prompt1 | model | parser

branch_chain = RunnableBranch(
    (lambda x: len(x.split())>300, prompt2 | model | parser),
    RunnablePassthrough()
)

final_chain = RunnableSequence(report_gen_chain, branch_chain)

print(final_chain.invoke({'topic':'Russia vs Ukraine'}))

The conflict between Russia and Ukraine has deep historical roots dating back to Ukraine's independence from Russia in 1991. Tensions escalated due to Ukraine's move towards European integration and NATO membership, leading to key events such as the annexation of Crimea in 2014 and the downing of Malaysia Airlines Flight MH17. The conflict remains unresolved with sporadic fighting in eastern Ukraine and strained relations between the two countries. Progress has been made, but lasting peace will require political will, compromise, and dialogue from both sides. Until then, the conflict poses a threat to regional security and stability.
